##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Klasifikasi Teks dengan teks yang telah diproses: Review Film

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat diTensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat source code di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Notebook ini mengklasifikasikan review dari film sebagai *positif* atau *negatif* menggunakan teks dari review tersebut. Ini merupakan contoh dari klasifikasi biner atau klasifikasi dengan dua kelas, sebuah permasalahan machine learning yang penting dan sering ditemukan.

Kita akan menggunakan data [IMDB dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews) yang terdiri atas 50,000 teks review film di [Internet Movie Database](https://www.imdb.com/). Data ini di bagi menjadi 25,000 review untuk proses training dan 25,000 review untuk testing. Data training dan testing sudah *seimbang*, artinya data mengandung jumlah positif dan negatif review yang sama.

Notebook ini menggunakan [tf.keras](https://www.tensorflow.org/guide/keras), sebuah API tingkat tinggi untuk membuat model di TensorFlow. Untuk tutorial klasifikasi teks yang lebih kompleks menggunakan `tf.keras`, lihat [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/).

## Pengaturan

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
from tensorflow import keras

!pip install tensorflow-datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np

print(tf.__version__)

<a id="download"></a>

## Unduh dataset IMDB

Dataset review film IMDB terdapat dalam package `tfds`. Teks yang terdapat dalam dataset ini sudah diproses terlebih dahulu sehingga review (urutan kata-kata) telah dikonversi menjadi urutan bilangan integer, dimana setiap integer merepresentasikan sebuah kata yang spesifik di dalam *dictionary*.

Kode berikut ini mengunduh dataset IMDB ke komputer (atau menggunakan *cached copy* apabila Anda telah mendownloadnya):

Untuk proses encode dari text Anda sendiri lihat [Loading text tutorial](../load_data/text.ipynb)

In [ ]:
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

<a id="encoder"></a>

## Mencoba Encoder

 `info` dataset ini meliputi enkoder dari teks (a `tfds.features.text.SubwordTextEncoder`).

In [ ]:
encoder = info.features['text'].encoder

In [ ]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Enkoder teks akan mengubah kembali bilangan integer menjadi string

In [ ]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

assert original_string == sample_string

Enkoder akan mengubah string dengan memecah string tersebut menjadi beberapa kata atau karakter apabila kata tersebut tidak terdapat di dalam *dictionary*. Jadi apabila string tersebut semakin menyerupai dataset, hasil *encode* dari string tersebut akan semakin singkat.

In [ ]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

## Eksplorasi data

Mari mencoba untuk memahami format dari data. Dataset sudah diproses terlebih dahulu sebelumnya: setiap data merupakan array dari integer yang merepresentasikan kata-kata yang terdapat di dalam review film.

Teks dari review sudah dikonversi ke integer, dimana setiap integer merepresentasikan sebuah kata spesifik di dalam *dictionary*.

Setiap label merupakan integer dengan value antara 0 atau 1, dimana 0 berarti review yang negatif, dan 1 berarti review yang positif.

Review pertama terlihat seperti ini:

In [ ]:
for train_example, train_label in train_data.take(1):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

struktur `info` memiliki enkoder/dekoder. Enkoder dapat digunakan untuk mengembalikan teks yang asli:

In [ ]:
encoder.decode(train_example)

## Mempersiapkan data untuk proses training

Anda menginginkan untuk membuat beberapa batch data training unutk model Anda. Review memiliki jumlah kata yang berbeda-beda, sehingga gunakan `padded_batch` untuk menambahkan nilai kosong dalam urutan kata ketika membuat batch:

In [ ]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32, train_data.output_shapes))

test_batches = (
    test_data
    .padded_batch(32, train_data.output_shapes))

Setiap batch akan memiliki bentuk `(batch_size, sequence_length)` karena padding yang ditambahkan bersifat dinamis sehingga setiap batch akan ditambahkan padding yang berbeda panjangnya:

In [ ]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)
  

## Membuat Model

Neural network dibuat dengan menumpuk layer-layer—hal ini membutuhkan dua keputusan arsitektural utama:

* Berapa layer yang akan digunakan dalam model?
* Berapa banyak *hidden units* yang akan digunakan tiap layer?

Dalam kasus ini, data input terdiri atas array berisi indeks dari kata-kata. Label yang akan diprediksi antara 0 atau 1. Mari buat model dengan cara "*Conitunous bag of words*" untuk permasalahan ini:

Peringatan: Model ini tidak menggunaan *masking*, sehingga *zero-padding* digunakan sebagai bagian dari input, sehingga jumlah padding dapat memengaruhi output dari model. Untuk memperbaiki hal ini, lihat [masking and padding guide](../../guide/keras/masking_and_padding).

In [ ]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.summary()

Layer-layer yang ditumpuk secara berurutan untuk membuat model klasifikasi:

1. Layer yang pertama adalah layer `Embedding`. Layer ini mengambil kosa kata dari integer yang sudah dienkode dan melihat vektor untuk masing-masing indeks kata. Vektor ini dipelajari bersamaan ketika model dilatih. Vektor ini menambahkan dimensi terhadap array output. Dimensi yang dihasilkan adalah: `(batch, sequence, embedding)`.
2. Selanjutnya, layer `GlobalAveragePooling1D` mengembalikan sebuah vektor output dengan panjang yang tetap untuk setiap data dengan merata-ratakan urutan dari dimensi. Hal ini menyebabkan model dapat mengatasi input dari panjang variabel dengan cara yang paling sederhana.
3. Vektor dengan panjang tetap ini kemudian melewati sebuah layer terhubung seluruhnya (`Dense`), layer ini memiliki 16 *hidden units*.
4. Layer terakhir adalah layer padat yang terhubung seluruhnya dengan satu node output. Menggunakan fungsi aktivasi `sigmoid`, nilai ini adalah bertipe float antara 0 dan 1, merepresentasikan probabilitas, atau tingkat percaya diri.

### Hidden units

Model diatas memiliki dua layer intermediate atau "hidden" layer, antara input dengan output. Jumlah output (unit, node, atau neuron) merupakan dimensi hasil representasi layer-layer tersebut. Dengan kata lain, kebebasan untuk menentukan jumlah network diperbolehkan ketika mempelajari representasi internal. 

Jika sebuah model memiliki hidden units lebih banyak (ruang dimensional representasi yang lebih tinggi), dan/atau memiliki jumlah layer lebih banyak, maka model tersebut dapat mempelajari representasi yang lebih kompleks. Akan tetapi, hal ini akan membuat model neural network menjadi lama dalam proses komputasi dan dapat menyebabkan model akan mempelajari pattern yang meningkatkan performansi pada data training tetapi tidak pada data test. Hal ini disebut dengan *overfitting*, dan kita akan menelusurinya lebih jauh.

### Loss function and optimizer

Sebuah model memerlukan *loss function* dan sebuah optimizer ketika proses training. Contoh ini merupakan kasus klasifikasi biner dan model menghasilkan probabilitas (sebuah layer dengan satu neuron menggunakan aktivasi sigmoid), kita akan menggunakan *loss function* berupa `binary_crossentropy`.

Ini bukanlah satu-satunya pilihan untuk *loss function*, Anda dapat, misalnya, memilih `mean_squared_error`. Akan tetapi, biasanya, `binary_crossentropy` lebih baik untuk mengatasi probabilitas—fungsi ini menghitung "jarak" antara distribusi probabilitas, atau dalam kasus ini, antara distribusi yang sebenarnya dengan prediksi.

Nanntinya, ketika kita menemui kasus regresi (misalnya, untuk memprediksi harga rumah), kita akan melihat bagaimana caranya menggunakan *loss function* lainnya yang disebut *mean squarred error*.

Sekarang, konfigurasi model menggunakan optimizer dan *loss function*:

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Melakukan training terhadap model

Model dilatih dengan memasukan objek `Dataset` ke fungsi fit dari model. Kemudian jumlah epochs diatur.

In [ ]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

## Model Evaluasi

Dan mari lihat bagaimana performansi dari model. Dua nilai akan dihasilkan. Loss (nilai yang merepresentasikan eror, semakin rendah nilainya semakin baik), dan akurasi.

In [ ]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Pendekatan yang cukup sederhana ini memperoleh akurasi sekitar 87%. Dengan cara yang lebih kompleks, model seharusya memperoleh akurasi mendekati 95%.

## Membuat grafik akurasi dan loss tiap waktu

`model.fit()` menghasilkan sebuah objek `History` yang mengandung sebuah *dictionary* dengan seluruh aktivitas yang terjadi ketika proses training berlangsung:

In [ ]:
history_dict = history.history
history_dict.keys()

Terdapat empat buah nilai: masing-masing satu nilai metriks (loss dan akurasi) yang terpantau ketika proses training dan validasi. Kita akan menggunakan nilai ini untuk membuat plot yang membandingkan antara nilai *loss* dari training dan validasi, juga plot yang membandingkan antara nilai akurasi training dan validasi:

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()


Dalam plot ini, titik merepresentasikan loss dan akurasi dari training, dan garis solid untuk loss dan akurasi dari validasi.

Perhatikan bahwa loss dari training *menurun* tiap epoch dan akurasinya *meningkat* tiap epoch. Hal ini yang kita kehendaki ketika menggunakan sebuah *gradient descent optimization*—optimisasi ini seharusnya meminimalkan jumlah yang diinginkan pada setiap iterasi.

Hal ini tidak terjadi untuk loss dan akurasi dari validasi—mereka terlihat mencapai titik tertinggi setelah dua puluh epoch. Hal ini merupakan contoh dari *overfitting*: model memiliki performansi yang lebih baik pada data training dibandingkan dengan performansinya terhadap data yang belum pernah dilihat sebelumnya. Setelah titik ini, model terlalu optimis dan mempelajari representasi yang sangat *spesifik* dari data train yang tidak *menggambarkan* data test.

Untuk kasus semacam ini, kita dapat mencegah *overfitting* dengan memberhentikan proses training setelah sekitar dua puluh epoch. Nantinya, Anda akan melihat bagaimana caranya melakukan hal ini secara otomatis menggunakan sebuah *callback*.